## Importing Libraries

In [1]:
import numpy as np

In [2]:
import tensorflow as tf


In [3]:
import tensorflow_datasets as tfds


## Data

In [4]:
mnist_dataset, mnist_info= tfds.load(name='mnist', with_info= True, as_supervised= True)

In [5]:
mnist_train, mnist_test= mnist_dataset['train'], mnist_dataset['test']

In [6]:
num_validation_samples= 0.1 * mnist_info.splits['train'].num_examples

In [7]:
num_validation_samples= tf.cast(num_validation_samples, dtype=tf.int64)

In [8]:
num_test_samples= mnist_info.splits['test'].num_examples

In [9]:
num_test_samples= tf.cast(num_test_samples, dtype=tf.int64)

### Scaling the data

In [10]:
def scale(image,label):
    image= tf.cast(image, tf.float32)
    image= image/255.
    return image,label

In [11]:
scaled_train_and_validation_data= mnist_train.map(scale)

In [12]:
test_data= mnist_test.map(scale)

### Shuffling the data

In [13]:
BUFFER_SIZE= 10000

In [14]:
shuffled_train_and_validation_data= scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [15]:
validation_data= shuffled_train_and_validation_data.take(num_validation_samples)

In [16]:
train_data= shuffled_train_and_validation_data.skip(num_validation_samples)

### Batching

In [17]:
BATCH_SIZE=100

In [18]:
train_data= train_data.batch(BATCH_SIZE)

In [19]:
validation_data= validation_data.batch(num_validation_samples)

In [20]:
test_data= test_data.batch(num_test_samples)

In [21]:
validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outlining the model

In [22]:
input_size=784

In [23]:
output_size=10

In [24]:
hidden_layer_size=50

In [25]:
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose the optimizer and loss function

In [26]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [27]:
NUM_EPOCHS=5

In [28]:
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.4047 - accuracy: 0.8863 - val_loss: 0.2172 - val_accuracy: 0.9342 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1803 - accuracy: 0.9475 - val_loss: 0.1648 - val_accuracy: 0.9498 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1399 - accuracy: 0.9587 - val_loss: 0.1385 - val_accuracy: 0.9608 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.1139 - accuracy: 0.9664 - val_loss: 0.1084 - val_accuracy: 0.9677 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0977 - accuracy: 0.9709 - val_loss: 0.1003 - val_accuracy: 0.9690 - 2s/epoch - 4ms/step


## Testing the model

In [29]:
test_loss, test_accuracy= model.evaluate(test_data)

1/1 [==============================] - 0s 323ms/step - loss: 0.1103 - accuracy: 0.9667


In [30]:
print('Test loss: {0:.2f} Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

Test loss: 0.11 Test accuracy: 96.67%
